In [1]:
import pandas as pd
import numpy as np
import snorkel
import matplotlib.pyplot as plt
from snorkel.labeling import labeling_function
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
import seaborn as sns

In [2]:
# read in the training data
df_train = pd.read_csv("data/processed-labels/training_set_seattle_4.csv")
df_train.head()

# read in gt.csv file
df_test = pd.read_csv('data/processed-labels/test_set_seattle_4.csv')
Y_test = df_test.verified.values
df_test.head()

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,...,clustered,count,false_positive,distance,tag_list,description,tag_count,way_type,intersection_distance,verified
0,9,CurbRamp,2.0,3TO1sxavsQLIliltuPh5yw,3,299.323669,-17.540178,180.368912,1.460938,e4ff325c-309b-4ea6-8555-8f973e662362,...,1.0,6.0,0,12.359854,1.0,0.0,1.0,tertiary,40.679454,1
1,12,SurfaceProblem,3.0,2zMHlmqFF6V2SHqueb1fZA,3,20.665178,-14.486608,89.751045,-1.658913,e4ff325c-309b-4ea6-8555-8f973e662362,...,0.0,3.0,0,1.758375,1.0,0.0,1.0,-1,-1.000000,1
2,20,CurbRamp,1.0,lk95zcW0edw2sZCB1rMPTA,3,341.866058,-12.957589,106.566940,-2.300713,e4ff325c-309b-4ea6-8555-8f973e662362,...,1.0,6.0,0,5.536850,0.0,0.0,0.0,residential,13.214736,1
3,22,CurbRamp,1.0,hYEH3dM1ydGeOvU7l906_A,3,210.609375,-13.984375,255.282394,0.888550,e4ff325c-309b-4ea6-8555-8f973e662362,...,1.0,4.0,0,11.320579,0.0,0.0,0.0,residential,27.025511,1
4,23,SurfaceProblem,4.0,hYEH3dM1ydGeOvU7l906_A,2,308.345978,-34.910713,255.282394,0.888550,e4ff325c-309b-4ea6-8555-8f973e662362,...,0.0,4.0,0,15.889878,1.0,0.0,1.0,-1,-1.000000,1


In [3]:
#store all column names in a list
cols = df_train.columns.tolist()
cols

['label_id',
 'label_type',
 'severity',
 'gsv_panorama_id',
 'zoom',
 'heading',
 'pitch',
 'photographer_heading',
 'photographer_pitch',
 'user_id',
 'lat',
 'lng',
 'geometry',
 'clustered',
 'count',
 'false_positive',
 'distance',
 'tag_list',
 'description',
 'tag_count',
 'way_type',
 'intersection_distance']

In [4]:
label_types = df_train ['label_type'].unique().tolist()
label_types

['CurbRamp',
 'Obstacle',
 'SurfaceProblem',
 'NoSidewalk',
 'NoCurbRamp',
 'Occlusion',
 'Other',
 'Signal',
 'Crosswalk']

Labeling Functions

In [5]:
# Define the label mappings for convenience
NOT_SURE = -1
WRONG = 0
CORRECT = 1

@labeling_function()
def intersection(x):
    if x["way_type"] =='residential' and (x["intersection_distance"] >= 70 or 0 < x["intersection_distance"] <= 10):
        return WRONG 
    elif x["way_type"] =='living_street' and (x["intersection_distance"] >= 70 or 0 < x["intersection_distance"] <= 10):   
        return WRONG
    else:
        return NOT_SURE

    
#check if the label is in the cluster
@labeling_function()
def clustered(x):
    if x["label_type"] =='NoSidewalk' and x["count"] <= 2:
        return WRONG
    elif x["label_type"] =='NoCurbRamp' and x["count"] <= 4:
        return WRONG 
    elif x["label_type"] =='CurbRamp' and x["count"] > 5:
        return CORRECT
    else:
        return NOT_SURE
    # return CORRECT if x['count'] >= 2 else NOT_SURE


#check if it is a severity 4 or higher
@labeling_function()
def severity(x):
    if x["label_type"] =='NoSidewalk' and x["severity"] < 3:
        return WRONG
    elif x["label_type"] =='NoCurbRamp' and x["severity"] <= 2:
        return WRONG
    elif x["label_type"] =='CurbRamp' and x["severity"] >= 4:
        return WRONG
    elif (x["label_type"] =='Obstacle' or x["label_type"] =='SurfaceProblem') and x["severity"] >= 4:
        return CORRECT
    else:
        return NOT_SURE
    return CORRECT if x["severity"] >= 4 else NOT_SURE


#check if the user zoomed in
@labeling_function()
def zoom(x):
    # x in this case is a row of the dataframe
    if x["zoom"] > 2:
        return CORRECT
    elif x["zoom"] == 1:
        return WRONG
    else:
        return NOT_SURE


#check if the user has put a tag
@labeling_function()
def tags(x):
    if x["label_type"] =='NoSidewalk' and x["tag_list"] == 1:
        return CORRECT
    elif x["label_type"] =='NoCurbRamp' and x["tag_list"] == 1:
        return CORRECT
    elif x["label_type"] =='NoCurbRamp' and x["tag_list"] == 1:
        return CORRECT
    else:
        return NOT_SURE
    return CORRECT if x['tag_list']  == 1 else NOT_SURE


#check if the user has put a comment
@labeling_function()
def description(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['description']  == 1 else NOT_SURE

@labeling_function()
def distance(x):
    # x in this case is a row of the dataframe
    if x["label_type"] =='NoSidewalk' and x["distance"] < 10:
        return WRONG
    elif x["label_type"] =='Obstacle' and x["distance"] < 10:
        return WRONG  
    else:
        return NOT_SURE

lfs = [
    intersection, 
    clustered,
    severity,
    zoom,
    tags,
    description,
    distance
    ]

In [6]:
applier = PandasLFApplier(lfs=lfs)
summary = pd.DataFrame(columns=['label_type', 'precision'])

for i in range(0,5):
    L_train = applier.apply(df=df_train[df_train['label_type'] == label_types[i]])
    L_test = applier.apply(df=df_test[df_test['label_type'] == label_types[i]])
    Y_test = df_test[df_test['label_type'] == label_types[i]].verified.values
    # print(f"Labeling function analysis for {label_types[i]}")
    # print(LFAnalysis(L=L_train, lfs=lfs).lf_summary())
    label_model = LabelModel(cardinality=2, verbose=True)
    label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)
    label_model_acc = label_model.score(L=L_test, Y=Y_test, metrics=["precision"],tie_break_policy="random")[
    "precision"
]
    # print(f"{'Label Model Accuracy for:':<25} {label_model_acc * 100:.1f}%")
    summary = summary.append({'label_type': label_types[i], 'precision': round(label_model_acc,4)}, ignore_index=True)

100%|██████████| 5333/5333 [00:00<00:00, 30672.87it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.247]
INFO:root:[100 epochs]: TRAIN:[loss=0.000]
INFO:root:[200 epochs]: TRAIN:[loss=0.000]
 55%|█████▍    | 274/500 [00:00<00:00, 2739.32epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.000]
INFO:root:[400 epochs]: TRAIN:[loss=0.000]
100%|██████████| 500/500 [00:00<00:00, 3115.26epoch/s]
INFO:root:Finished Training
/var/folders/zm/x5dzdlrs5mlbxrm7z5g6p05r0000gn/T/ipykernel_12719/2083352784.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append({'label_type': label_types[i], 'precision': round(label_model_acc,4)}, ignore_index=True)
100%|██████████| 2909/2909 [00:00<00:00, 32739.86it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epoc

In [44]:
summary

,label_type,precision
0,CurbRamp,0.9515
1,Obstacle,0.6491
2,SurfaceProblem,0.9003
3,NoSidewalk,0.9545
4,NoCurbRamp,0.9154
